In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.request

import time
import pandas as pd
import scipy as sc
import numpy as np

import statsmodels.formula.api as sm

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6) 
from bs4 import UnicodeDammit


In [2]:
grades=pd.read_csv('grade_crosstab_data.csv', encoding="utf8")
i=0
hc=grades['sumHeadcount']
while i <= 1696:
    if hc[i]== 'ds':
        grades=grades.drop(i, axis=0)
        
    i+=1
grades=grades.reset_index(drop=True)
grades

,Catalog Num,Course Level,Grade Group,Section,Subject,sumHeadcount
0,1050,1000-1999,Other,1,ASTR - Astronomy,2
1,1050,1000-1999,W,1,ASTR - Astronomy,2
2,1050,1000-1999,E,1,ASTR - Astronomy,6
3,1050,1000-1999,D,1,ASTR - Astronomy,6
4,1050,1000-1999,C,1,ASTR - Astronomy,26
5,1050,1000-1999,B,1,ASTR - Astronomy,40
6,1050,1000-1999,A,1,ASTR - Astronomy,30
7,1060,1000-1999,W,1,ASTR - Astronomy,4
8,1060,1000-1999,E,1,ASTR - Astronomy,12
9,1060,1000-1999,D,1,ASTR - Astronomy,6


In [43]:
i=1
n=2
gradesclean=pd.DataFrame([],columns=['Num', 'Section', 'Subject', 'A', 'B', 'C', 'D', 'E', 'W', 'Other'])
gradesclean=gradesclean.append({'Num':1050, 'Section':1, 'Subject':'ASTR - Astronomy', 'A':0, 'B':0, 'C':0, 'D':0,
                                'E':0, 'W':0, 'Other':2}, ignore_index=True)
while i<1636:
    if grades['Subject'].iloc[i]==gradesclean['Subject'].iloc[-1]:
        if grades['Catalog Num'].iloc[i]==gradesclean['Num'].iloc[-1]:
            fail=0
            for m in range(1,n):
                if grades['Section'].iloc[i]==gradesclean['Section'].iloc[-m]:
                    gradesclean[grades['Grade Group'].iloc[i]].iat[-m]=grades['sumHeadcount'].iloc[i]
                    break
                fail+=1
            if fail==n-1:
                n+=1
                gradesclean=gradesclean.append({'Num':grades['Catalog Num'].iloc[i],
                                                'Section':grades['Section'].iloc[i],
                                                'Subject':grades['Subject'].iloc[i], 'A':0, 'B':0,
                                                'C':0,'D':0, 'E':0, 'W':0, 'Other':0}, ignore_index=True)
                gradesclean[grades['Grade Group'].iloc[i]].iat[-1]=grades['sumHeadcount'].iloc[i]
        else:
            n=2
            gradesclean=gradesclean.append({'Num':grades['Catalog Num'].iloc[i], 'Section':grades['Section'].iloc[i],
                                            'Subject':grades['Subject'].iloc[i], 'A':0, 'B':0,
                                            'C':0,'D':0, 'E':0, 'W':0, 'Other':0}, ignore_index=True)
            gradesclean[grades['Grade Group'].iloc[i]].iat[-1]=grades['sumHeadcount'].iloc[i]
    else:
        n=2
        gradesclean=gradesclean.append({'Num':grades['Catalog Num'].iloc[i], 'Section':grades['Section'].iloc[i],
                                        'Subject':grades['Subject'].iloc[i], 'A':0, 'B':0,
                                        'C':0,'D':0, 'E':0, 'W':0, 'Other':0}, ignore_index=True)
        gradesclean[grades['Grade Group'].iloc[i]].iat[-1]=grades['sumHeadcount'].iloc[i]
    i+=1
gradesclean

,Num,Section,Subject,A,B,C,D,E,W,Other
0,1050,1,ASTR - Astronomy,30,40,26,6,6,2,2
1,1060,1,ASTR - Astronomy,11,9,22,6,12,4,0
2,1080,1,ASTR - Astronomy,15,20,11,8,2,2,8
3,2500,1,ASTR - Astronomy,12,7,1,0,0,0,0
4,4060,1,ASTR - Astronomy,9,1,0,0,0,0,0
5,5580,1,ASTR - Astronomy,2,3,0,0,0,0,1
6,1006,1,BIOL - Biology,39,24,6,1,1,2,0
7,1010,1,BIOL - Biology,37,28,7,1,9,7,2
8,1030,302,BIOL - Biology,5,2,2,1,1,1,0
9,1030,301,BIOL - Biology,5,4,1,1,3,0,0
